In [ ]:

import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split,cross_val_score, KFold	
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, matthews_corrcoef
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV





    


bank_train = read_csv('bank_train.csv')
bank_test = read_csv('bank_test.csv')
bank = bank_train.append(bank_test)


#  Split dataset into k consecutive folds
def dfSplit(train_data, nfold):
    y=train_data['y']
    x=train_data.drop('y',axis=1)
    
    #This list stores 5 training dataframes which contain only features
    train_x_sets=[]
    #This list stores 5 training dataframes which contain only the target
    train_y_sets=[]
    
    #This list stores 5 testing/validation dataframes which contain only features
    test_x_sets=[]
    #This list stores 5 testing/validation dataframes which contain only the dataset
    test_y_sets=[]
    
    kf = KFold(n_splits=nfold,random_state=111)
    
    for train_ind, valid_ind in kf.split(train_data):
        
        x_train, y_train=x.iloc[train_ind], y.iloc[train_ind]
        x_valid, y_valid=x.iloc[valid_ind], y.iloc[valid_ind]
        
        train_x_sets.append(x_train)
        train_y_sets.append(y_train)
        
        test_x_sets.append(x_valid)
        test_y_sets.append(y_valid)
        print('train:' ,train_ind,"test:", valid_ind)
        #print((x_train).shape,(x_valid).shape)
        #print((y_train).shape,(y_valid).shape)
    return train_x_sets, train_y_sets, test_x_sets, test_y_sets

# def overSampling(train_x_sets, train_y_sets):
#     for 
#     smote_enn = SMOTEENN(random_state=0)
#     X_train_i,Y_train_i = smote_enn.fit_sample(X_train_i,Y_train_i)
    
    



In [ ]:
        



#5-fold Cross validation to find best n_neighbors
#Return a model with the best parameter
def mlpClassifier(data):
    
    #five_fold = KFold(n_splits=5, shuffle=False,random_state=111)
# =============================================================================
#     params =  {"hidden_layer_sizes": np.arange(2, 10),
#                              "solver": ['adam', 'sgd'],
#                              "max_iter": [500],
#                              "verbose": [True],
#                              'shuffle' :[False],
#                              'early_stopping': [True]
#                              }
#     
#     
# =============================================================================
 
   
# =============================================================================
#     params =  {"hidden_layer_sizes": [(100,), (100, 30)],
#                               "solver": ['adam', 'sgd'],
#                               "max_iter": [500],
#                               "verbose": [True],
#                               'shuffle' :[False],
#                               'early_stopping': [True]
#                               }
# 
# =============================================================================


    params = {'hidden_layer_sizes': [(100,50,25), (50,100,50), (100,50),(100,),(50,25)],
              'activation': ['relu'],
              'solver': ['sgd', 'adam'],
              'max_iter': [500],
              "verbose": [True],
              'random_state':[111],
              'early_stopping': [True]
              }

    #{'hidden_layer_sizes':np.arange(2, 10), 'max_iter': [500],'shuffle' :[False]}
    mlp = GridSearchCV(MLPClassifier(), params, cv=5,n_jobs=6)
    mlp.fit(data.drop('y',axis=1), data['y'])
#选出最优参数    
    best_parameters = mlp.best_estimator_.get_params()  
    print(best_parameters)
    print(mlp.get_params().keys())
    print(mlp.best_params_)
    
    return mlp











        
        
    

In [ ]:

from imblearn.combine import SMOTEENN


# =============================================================================
# #Perform 5-fold valiadation to get traning performance by MCC
# def Mcc_Cv(model,bk_train_x_sets, bk_train_y_sets,bk_test_x_sets, bk_test_y_sets):
#     mcc_list=[]
#     for i in range(len(bk_train_x_sets)):
#             model.fit(bk_train_x_sets[i],bk_train_y_sets[i])
#             prediction = model.predict(bk_test_x_sets[i])
#             mcc = matthews_corrcoef(bk_test_y_sets[i], prediction)
#             mcc_list.append(mcc)
#             print('For split={}, MCC is {}'.format(i+1,mcc_list[i]))
#     print()
#     print('mean MCC is',np.mean(mcc_list))
# =============================================================================
    
def Mcc_Cv(model,bk_train_x_sets, bk_train_y_sets,bk_test_x_sets, bk_test_y_sets):
    mcc_list=[]
    for i in range(len(bk_train_x_sets)):
      X_train_i = bk_train_x_sets[i]
      Y_train_i = bk_train_y_sets[i]
      X_test_i = bk_test_x_sets[i]
      Y_test_i = bk_test_y_sets[i]
      smote_enn  = SMOTEENN(random_state=111)
      X_train_i,Y_train_i = smote_enn.fit_sample(X_train_i,Y_train_i)
      model.fit(X_train_i,Y_train_i )
      prediction = model.predict(X_test_i)
      mcc = matthews_corrcoef(Y_test_i, prediction)
      mcc_list.append(mcc)
      print('For split={}, MCC is {}'.format(i+1,mcc_list[i]))
    print()
    print('mean MCC is',np.mean(mcc_list))    




In [ ]:

#run model
bank_train_x_sets, bank_train_y_sets,bank_test_x_sets, bank_test_y_sets = dfSplit(bank,5)

mlp_model = model= MLPClassifier(activation= 'relu', early_stopping= True, hidden_layer_sizes= (50, 100, 50), max_iter= 500, random_state= 111, solver= 'adam', verbose= True)

Mcc_Cv(mlp_model,bank_train_x_sets, bank_train_y_sets,bank_test_x_sets, bank_test_y_sets)    
        

C:\Users\33536\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


train: [ 9043  9044  9045 ... 45208 45209 45210] test: [   0    1    2 ... 9040 9041 9042]
train: [    0     1     2 ... 45208 45209 45210] test: [ 9043  9044  9045 ... 18082 18083 18084]
train: [    0     1     2 ... 45208 45209 45210] test: [18085 18086 18087 ... 27124 27125 27126]
train: [    0     1     2 ... 45208 45209 45210] test: [27127 27128 27129 ... 36166 36167 36168]
train: [    0     1     2 ... 36166 36167 36168] test: [36169 36170 36171 ... 45208 45209 45210]
Iteration 1, loss = 1.62842055
Validation score: 0.872402
Iteration 2, loss = 0.98420497
Validation score: 0.879257
Iteration 3, loss = 0.67083018
Validation score: 0.889429
Iteration 4, loss = 0.71549273
Validation score: 0.886112
Iteration 5, loss = 0.73597743
Validation score: 0.892968
Iteration 6, loss = 0.73749271
Validation score: 0.888324
Iteration 7, loss = 0.59425182
Validation score: 0.820433
Iteration 8, loss = 0.43237749
Validation score: 0.888324
Iteration 9, loss = 0.36195190
Validation score: 0.847192

Iteration 26, loss = 0.15610916
Validation score: 0.908650
Iteration 27, loss = 0.13609793
Validation score: 0.952708
Iteration 28, loss = 0.17726259
Validation score: 0.958367
Iteration 29, loss = 0.13960950
Validation score: 0.950687
Iteration 30, loss = 0.15816068
Validation score: 0.931083
Iteration 31, loss = 0.14859861
Validation score: 0.936338
Iteration 32, loss = 0.15692467
Validation score: 0.934721
Iteration 33, loss = 0.13981723
Validation score: 0.952102
Iteration 34, loss = 0.15107906
Validation score: 0.955335
Iteration 35, loss = 0.12589997
Validation score: 0.957559
Iteration 36, loss = 0.11790566
Validation score: 0.955740
Iteration 37, loss = 0.11960687
Validation score: 0.951698
Iteration 38, loss = 0.14246122
Validation score: 0.945432
Iteration 39, loss = 0.12763232
Validation score: 0.946241
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
For split=2, MCC is 0.5152934801350694
Iteration 1, loss = 1.73382421
Validation 

Iteration 34, loss = 0.11796516
Validation score: 0.937930
Iteration 35, loss = 0.12722731
Validation score: 0.956126
Iteration 36, loss = 0.13240993
Validation score: 0.935099
Iteration 37, loss = 0.11526408
Validation score: 0.940962
Iteration 38, loss = 0.11402619
Validation score: 0.942984
Iteration 39, loss = 0.13477030
Validation score: 0.956733
Iteration 40, loss = 0.11826089
Validation score: 0.937323
Iteration 41, loss = 0.12377131
Validation score: 0.956126
Iteration 42, loss = 0.12089208
Validation score: 0.951880
Iteration 43, loss = 0.13333568
Validation score: 0.947634
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
For split=5, MCC is 0.510684719111293

mean MCC is 0.5139248464522098
